In [ ]:
%spark
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.streaming._
import org.apache.spark.sql.types._
val spark = SparkSession.builder.appName("streaming Application").config("spark.sql.catalogImplementation","in-memory").getOrCreate()

In [6]:
%spark
import org.apache.spark.sql.*;
import org.apache.spark.sql.streaming.StreamingQuery;
import org.apache.spark.sql.streaming.Trigger;
import static org.apache.spark.sql.functions.col;
import static org.apache.spark.sql.avro.functions.*;

val jsonFormatSchema = new String(Files.readAllBytes(Paths.get("./src/main/resources/user.avsc")))


SparkSession spark = SparkSession
  .builder()
  .appName("Streaming Application")
  .getOrCreate();


Dataset<Row> df = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "b-3.demo-cluster-1.25yk5g.c8.kafka.eu-west-1.amazonaws.com:9092,b-1.demo-cluster-1.25yk5g.c8.kafka.eu-west-1.amazonaws.com:9092,b-2.demo-cluster-1.25yk5g.c8.kafka.eu-west-1.amazonaws.com:9092").option("subscribe", "MSKTutorialTopic").option("startingOffsets", "earliest").load()
Dataset<Row> output = df.select(from_avro(col("value"), jsonFormatSchema).as("record")).select("record.*")
StreamingQuery query = output.writeStream
  .format("csv")
  .option("checkpointLocation", "/tmp/test")
  .option("path", "/output/")
  .trigger(Trigger.ProcessingTime("1 seconds"))
  .start()
query.awaitTermination()